In [ ]:
! pip install tensorflow

In [ ]:
!pip install keras

In [ ]:
! pip install seaborn

In [ ]:
! pip show keras

Name: keras
Version: 2.12.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: tensorflow


In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

import tensorflow_hub as hub

In [ ]:
test_dir='/content/gdrive/MyDrive/DATA/test_set/test_set'
cats_test='/content/gdrive/MyDrive/DATA/test_set/test_set/cats'
dog_test='/content/gdrive/MyDrive/DATA/test_set/test_set/dogs'

In [ ]:
train_dir='/content/gdrive/MyDrive/DATA/training_set/training_set'
cats_train='/content/gdrive/MyDrive/DATA/training_set/training_set/cats'
dogs_train='/content/gdrive/MyDrive/DATA/training_set/training_set/dogs'

In [ ]:
data_dir=('/content/gdrive/MyDrive/DATA')

In [ ]:
data_gen = ImageDataGenerator(rescale = 1/255., validation_split = 0.2)

train_data = data_gen.flow_from_directory(data_dir,
                                          target_size = (224, 224),
                                          batch_size = 32,
                                          subset = 'training',
                                          class_mode = 'binary')
val_data = data_gen.flow_from_directory(data_dir,
                                        target_size = (224, 224),
                                        batch_size = 32,
                                        subset = 'validation',
                                        class_mode = 'binary')

Found 8023 images belonging to 2 classes.
Found 2005 images belonging to 2 classes.


In [ ]:
images, labels = train_data.next()
len(images), len(labels), images[0].shape

(32, 32, (224, 224, 3))

In [ ]:
train_data, val_data

(<keras.preprocessing.image.DirectoryIterator at 0x7f4144ce3d60>,
 <keras.preprocessing.image.DirectoryIterator at 0x7f41d8a21b70>)

In [ ]:
model = tf.keras.Sequential([
            tf.keras.Input(shape = images[0].shape),
            tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2), strides=2),
            tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2), strides=2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation = 'relu'),
            tf.keras.layers.Dense(2, activation = 'softmax'),
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 200704)            0         
                                                                 
 dense (Dense)               (None, 128)               2

In [ ]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor = 'val_acc', patience = 3, factor = 0.5, min_lr = 0.00001)

In [ ]:
classifier = model.fit(train_data, batch_size = 32,
                                 epochs = 12,
                                 validation_data = val_data,
                                 callbacks = [reduce_lr])

Epoch 1/12
251/251 [==============================] - ETA: 0s - loss: 0.6825 - accuracy: 0.7921

251/251 [==============================] - 1456s 6s/step - loss: 0.6825 - accuracy: 0.7921 - val_loss: 0.5130 - val_accuracy: 0.7985 - lr: 0.0010
Epoch 2/12
251/251 [==============================] - ETA: 0s - loss: 0.5091 - accuracy: 0.7982

251/251 [==============================] - 49s 195ms/step - loss: 0.5091 - accuracy: 0.7982 - val_loss: 0.5158 - val_accuracy: 0.7985 - lr: 0.0010
Epoch 3/12
251/251 [==============================] - ETA: 0s - loss: 0.4967 - accuracy: 0.7987

251/251 [==============================] - 50s 198ms/step - loss: 0.4967 - accuracy: 0.7987 - val_loss: 0.5128 - val_accuracy: 0.7985 - lr: 0.0010
Epoch 4/12
251/251 [==============================] - ETA: 0s - loss: 0.4353 - accuracy: 0.8174

251/251 [==============================] - 51s 204ms/step - loss: 0.4353 - accuracy: 0.8174 - val_loss: 0.6188 - val_accuracy: 0.7835 - lr: 0.0010
Epoch 5/12
251/251 [==============================] - ETA: 0s - loss: 0.3072 - accuracy: 0.8719

...

251/251 [==============================] - 51s 204ms/step - loss: 0.0608 - accuracy: 0.9812 - val_loss: 1.8981 - val_accuracy: 0.7107 - lr: 0.0010
Epoch 9/12
251/251 [==============================] - ETA: 0s - loss: 0.0433 - accuracy: 0.9867

251/251 [==============================] - 50s 199ms/step - loss: 0.0433 - accuracy: 0.9867 - val_loss: 2.3801 - val_accuracy: 0.7297 - lr: 0.0010
Epoch 10/12
251/251 [==============================] - ETA: 0s - loss: 0.0298 - accuracy: 0.9923

251/251 [==============================] - 52s 205ms/step - loss: 0.0298 - accuracy: 0.9923 - val_loss: 2.5205 - val_accuracy: 0.7062 - lr: 0.0010
Epoch 11/12
251/251 [==============================] - ETA: 0s - loss: 0.0189 - accuracy: 0.9953

251/251 [==============================] - 49s 197ms/step - loss: 0.0189 - accuracy: 0.9953 - val_loss: 2.7675 - val_accuracy: 0.6848 - lr: 0.0010
Epoch 12/12
251/251 [==============================] - ETA: 0s - loss: 0.0411 - accuracy: 0.9883

251/251 [==============================] - 51s 204ms/step - loss: 0.0411 - accuracy: 0.9883 - val_loss: 2.6253 - val_accuracy: 0.7312 - lr: 0.0010


In [ ]:
model.evaluate(val_data)

63/63 [==============================] - 10s 157ms/step - loss: 2.6253 - accuracy: 0.7312


[2.6253178119659424, 0.7311720848083496]

In [ ]:
model.save('model.h5')

NameError: ignored

In [ ]:
loaded_model = tf.keras.models.load_model('/content/model.h5')

In [ ]:
def preprocess_images(path):
    img = image.load_img(path, target_size=(224, 224))

    img_array = image.img_to_array(img)

    img_array = np.expand_dims(img_array, axis=0)

    img_preprocessed = img_array / 255.0

    return img_preprocessed


In [ ]:
img_path = "/content/gdrive/MyDrive/DATA/test_set/test_set/cats/cat.4002.jpg"
img = preprocess_images(img_path)
prediction = loaded_model.predict(img)

predicted_class_index = np.argmax(prediction)

class_labels = ['cat', 'not_cat']
predicted_class_label = class_labels[predicted_class_index]

print("Predicted class:", predicted_class_label)

1/1 [==============================] - 0s 58ms/step
Predicted class: cat
